# Table of Contents

1. Imports
2. Web Scraping Premier League Data
    - Scraping one team from the current season
    - Scraping at a larger scale over multiple seasons
3. Data Inspection and Clean-Up
4. Data Analysis and Visualization

## Imports

In [558]:
pip install chart_studio # To ensure my charts can be used on powerpoint!

     ---------------------------------------- 64.4/64.4 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [561]:
import requests  # Importing the libray I need
from bs4 import BeautifulSoup # We need this to parse through the HTML in an understandable way
import pandas as pd
import numpy as np
import time #To stagger our scraping frequency
import chart_studio

username = 'Kvothe_v1'
api_key = '4usd06VNwa8moSIVWdRF'
chart_studio.tools.set_credentials_file(username, api_key=api_key)
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objs as go


## Web Scraping Premier League Data

In [17]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats' ##The page I'll be scraping

In [18]:
data = requests.get(standings_url) 
#This makes a request to the server, giving me permission to download the html of this page

In [22]:
soup = BeautifulSoup(data.text)

In [23]:
standings_table = soup.select('table.stats_table')[0] 
#This gives us the table tag with stats table we want exactly
#Index of [0] because I only need 

In [24]:
links = standings_table.find_all('a')
links 
#We need to find all the html ('<a>' tags) within the standings_table as they hold the 
#links to information in the standings_table

[<a href="/en/squads/18bb7c10/Arsenal-Stats">Arsenal</a>,
 <a href="/en/matches/1b35bc71/Arsenal-Manchester-United-January-22-2023-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>,
 <a href="/en/matches/07fc721a/Everton-Arsenal-February-4-2023-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">L</a>,
 <a href="/en/matches/7caa56bc/Arsenal-Brentford-February-11-2023-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">D</a>,
 <a href="/en/matches/b9da01e6/Arsenal-Manchester-City-February-15-2023-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">L</a>,
 <a href="/en/matches/e50bfcf8/Aston-Villa-Arsenal-February-18-2023-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>,
 <a href="/en/players/bc7dc64d/Bukayo-Saka">Bukayo Saka</a>,
 <a href="/en/players/466fb2c5/Aaron-Ramsdale">Aaron Ramsdale</a

In [25]:
links = [l.get("href") for l in links] 
#list comprehension that extracts the href attribute values of all HTML anchor elements in the links list 
#in the same order as the original links list.
links

['/en/squads/18bb7c10/Arsenal-Stats',
 '/en/matches/1b35bc71/Arsenal-Manchester-United-January-22-2023-Premier-League',
 '/en/matches/07fc721a/Everton-Arsenal-February-4-2023-Premier-League',
 '/en/matches/7caa56bc/Arsenal-Brentford-February-11-2023-Premier-League',
 '/en/matches/b9da01e6/Arsenal-Manchester-City-February-15-2023-Premier-League',
 '/en/matches/e50bfcf8/Aston-Villa-Arsenal-February-18-2023-Premier-League',
 '/en/players/bc7dc64d/Bukayo-Saka',
 '/en/players/466fb2c5/Aaron-Ramsdale',
 '/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/matches/7337f90a/Manchester-City-Wolverhampton-Wanderers-January-22-2023-Premier-League',
 '/en/matches/8a923619/Tottenham-Hotspur-Manchester-City-February-5-2023-Premier-League',
 '/en/matches/18fcd595/Manchester-City-Aston-Villa-February-12-2023-Premier-League',
 '/en/matches/b9da01e6/Arsenal-Manchester-City-February-15-2023-Premier-League',
 '/en/matches/7f12d9aa/Nottingham-Forest-Manchester-City-February-18-2023-Premier-League',
 '/en/pla

In [26]:
squads = [l for l in links if '/squads/' in l] 
#list comprehension that filters the links list to include only the HTML anchor elements that contain the string 
#'/squads/' in their href attribute
#This allows us to pull only the squad links as we need the stats for each team
squads

['/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/a2d435b3/Leicester-City-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/5bfb9659/Leeds-United-Stats',
 '/en/squads/33c895d4/Southampton-Stats']

In [27]:
team_urls = [f"https://fbref.com{l}" for l in squads] #completes the url for each team

In [28]:
team_urls

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com

### Scraping Data for One Team from the Current Season

In [30]:
team_url = team_urls[0] #To test my following code I'll use Arsenal's stats, the first index

In [31]:
data = requests.get(team_url) #To get permission from the server to downlaod this link for Arsenal

In [32]:
matches = pd.read_html(data.text, match="Scores & Fixtures") 
# Creates a list of DataFrame objects using the specific match parameter

In [33]:
matches[0]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,1.0,1.2,44.0,25286.0,Martin Ødegaard,4-3-3,Anthony Taylor,Match Report,NaN
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.7,0.5,50.0,60033.0,Martin Ødegaard,4-3-3,Darren England,Match Report,NaN
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.3,0.3,57.0,10423.0,Martin Ødegaard,4-3-3,Craig Pawson,Match Report,NaN
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.6,0.8,71.0,60164.0,Martin Ødegaard,4-3-3,Jarred Gillett,Match Report,NaN
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.4,0.4,59.0,60012.0,Martin Ødegaard,4-3-3,Robert Jones,Match Report,NaN
5,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,1.3,1.5,60.0,73431.0,Martin Ødegaard,4-3-3,Paul Tierney,Match Report,NaN
6,2022-09-08,18:45,Europa Lg,Group stage,Thu,Away,W,2.0,1.0,ch Zürich,2.3,1.2,69.0,17070.0,Granit Xhaka,4-3-3,Mohammed Al Hakim,Match Report,NaN
7,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3.0,0.0,Brentford,1.5,0.5,63.0,17122.0,Granit Xhaka,4-3-3,David Coote,Match Report,NaN
8,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3.0,1.0,Tottenham,2.4,1.6,64.0,60278.0,Martin Ødegaard,4-3-3,Anthony Taylor,Match Report,NaN
9,2022-10-06,20:00,Europa Lg,Group stage,Thu,Home,W,3.0,0.0,no Bodø/Glimt,3.4,0.8,50.0,59724.0,Granit Xhaka,4-3-3,Harm Osmers,Match Report,NaN


In [34]:
soup = BeautifulSoup(data.text) #To parse through the data about Arsenal's stats

In [35]:
shots = soup.find_all('a') #To get all the 'a' tags as they house the links we need

In [36]:
shots = [s.get("href") for s in shots] #To get the "href" of each 'a' tag

In [37]:
shots

['https://fbref.com/pt/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/fr/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/it/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/de/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/es/squads/18bb7c10/Arsenal-Stats',
 'https://www.sports-reference.com/?utm_source=fb&utm_medium=sr_xsite&utm_campaign=2023_01_srnav',
 'https://www.baseball-reference.com/?utm_source=fb&utm_medium=sr_xsite&utm_campaign=2023_01_srnav',
 'https://www.pro-football-reference.com/?utm_source=fb&utm_medium=sr_xsite&utm_campaign=2023_01_srnav',
 'https://www.sports-reference.com/cfb/',
 'https://www.basketball-reference.com/?utm_source=fb&utm_medium=sr_xsite&utm_campaign=2023_01_srnav',
 'https://www.sports-reference.com/cbb/',
 'https://www.hockey-reference.com/?utm_source=fb&utm_medium=sr_xsite&utm_campaign=2023_01_srnav',
 'https://fbref.com/?utm_source=fb&utm_medium=sr_xsite&utm_campaign=2023_01_srn

In [39]:
shots = [t for t in shots if t and 'all_comps/shooting/' in t] 
#This finds all the shooting links in the list of 4 "href"s but returns duplicate values.

In [40]:
data = requests.get(f"https://fbref.com{shots[0]}") 
#To complete the links for shooting stats

In [42]:
shooting = pd.read_html(data.text, match="Shooting")[0] #Returns Arsenal's shooting data as a table

In [43]:
shooting.head() # returns the first 5 rows

For Arsenal                                                              \
         Date   Time            Comp        Round  Day Venue Result GF GA   
0  2022-08-05  20:00  Premier League  Matchweek 1  Fri  Away      W  2  0   
1  2022-08-13  15:00  Premier League  Matchweek 2  Sat  Home      W  4  2   
2  2022-08-20  17:30  Premier League  Matchweek 3  Sat  Away      W  3  0   
3  2022-08-27  17:30  Premier League  Matchweek 4  Sat  Home      W  2  1   
4  2022-08-31  19:30  Premier League  Matchweek 5  Wed  Home      W  2  1   

                   ... Standard               Expected                    \
         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  Crystal Palace  ...     14.6  1.0  0     0      1.0  1.0    0.10  0.0   
1  Leicester City  ...     13.0  0.0  0     0      2.7  2.7    0.16  1.3   
2     Bournemouth  ...     14.8  0.0  0     0      1.3  1.3    0.10  1.7   
3          Fulham  ...     15.5  1.0  0     0      2.6  2.6    0.12 -0.6   
4     Aston Villa  ...     16.3  1.0  0     0      2.4  2.4    0.12 -0.4   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     0.0        Match Report  
1     1.3        Match Report  
2     1.7        Match Report  
3    -0.6        Match Report  
4    -0.4        Match Report  

[5 rows x 26 columns]

In [44]:
shooting.columns = shooting.columns.droplevel()
#To get rid of the multi-level index which prevents us from making calculations

In [45]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2,0,Crystal Palace,...,14.6,1.0,0,0,1.0,1.0,0.10,0.0,0.0,Match Report
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,2,Leicester City,...,13.0,0.0,0,0,2.7,2.7,0.16,1.3,1.3,Match Report
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3,0,Bournemouth,...,14.8,0.0,0,0,1.3,1.3,0.10,1.7,1.7,Match Report
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,15.5,1.0,0,0,2.6,2.6,0.12,-0.6,-0.6,Match Report
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,16.3,1.0,0,0,2.4,2.4,0.12,-0.4,-0.4,Match Report


In [46]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
## To merge the shooting stats table with the match stats table on the "Date" column

In [47]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,4-3-3,Anthony Taylor,Match Report,NaN,10,2,14.6,1.0,0,0
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,4-3-3,Darren England,Match Report,NaN,19,7,13.0,0.0,0,0
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,4-3-3,Craig Pawson,Match Report,NaN,14,6,14.8,0.0,0,0
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,4-3-3,Jarred Gillett,Match Report,NaN,22,8,15.5,1.0,0,0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,4-3-3,Robert Jones,Match Report,NaN,22,8,16.3,1.0,0,0


In [48]:
team_data.shape

(32, 25)

### Scraping at a Larger Scale over Multiple Seasons

In [49]:
years = list(range(2022,2020, -1)) #Creates a list starting from 2023 and ending at 2022

In [53]:
years 

[2022, 2021]

In [54]:
all_matches = []

In [52]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [55]:
for year in years: #To gather information based on the years in our list
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    #Selects the first HTML table element with the CSS class name "stats_table"
    links = [l.get("href") for l in standings_table.find_all('a')]
    #List comprehension to get the 'href' in my standings_table
    squads = [l for l in links if '/squads/' in l]
    #List comprehension to get the links with 'squads' in it
    team_urls = [f"https://fbref.com{l}" for l in squads]
    #list comprehension to turn the links from relative to absolute links
    previous_season = soup.select("a.prev")[0].get("href")
    #To get the Href link for the previous season
    standings_url = f"http://fbref.com/{previous_season}"
    #To complete the url
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        #To remove any extra characters from the team url and returns the team name 
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        #This gives me the first table (Scores and Fixtures as opposed to multiple tables!)
        
        soup = BeautifulSoup(data.text)
        tot_shots = [s.get("href") for s in soup.find_all('a')]
        total_shots = [t for t in tot_shots if t and 'all_comps/shooting/' in t]
        #To get the shooting stats table url
        data = requests.get(f"https://fbref.com{total_shots[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        #To get the table for Shooting stats
        shooting.columns = shooting.columns.droplevel()
        #This removes the top row as it is unnecesary
        try:
            team_data = matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
        #This code takes a subset of the shooting DataFrame, containing 
        #only the "Date" and certain shooting statistics columns, and merges it with the matches 
        
        except ValueError:
            continue
         #If the merge fails for any reason, such as if the "Date" or "shooting stats do not exist in one or both of the DataFrames, 
         #a ValueError will be raised.
            
            
        team_data = team_data[team_data["Comp"] == "Premier League"]
        #Only shows values in the "Comp" column where Premier League is the "Competition"
        
        team_data["Season"] = year
        #Specifies what season a match strat is from
        team_data["Team"] = team_name
        #This shows what team the data refers to in the table
        all_matches.append(team_data)
        time.sleep(4)
    

In [56]:
matches_df = pd.concat(all_matches)
#This concatenates my list of DataFrames horizontally into one single DataFrame

In [556]:
matches_df.head(50)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal
5,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,...,Match Report,NaN,16.0,3.0,18.6,1.0,0.0,0.0,2022,Arsenal
7,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3.0,0.0,Brentford,...,Match Report,NaN,13.0,7.0,18.2,0.0,0.0,0.0,2022,Arsenal
8,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3.0,1.0,Tottenham,...,Match Report,NaN,22.0,9.0,18.6,1.0,0.0,0.0,2022,Arsenal
10,2022-10-09,16:30,Premier League,Matchweek 10,Sun,Home,W,3.0,2.0,Liverpool,...,Match Report,NaN,10.0,6.0,14.3,0.0,1.0,1.0,2022,Arsenal
12,2022-10-16,14:00,Premier League,Matchweek 11,Sun,Away,W,1.0,0.0,Leeds United,...,Match Report,NaN,9.0,4.0,16.6,1.0,0.0,0.0,2022,Arsenal


In [60]:
matches_df.to_csv("matches_premier.csv")
# This converts my DataFrame to a CSV file

## Data Inspection and Clean-up

In [89]:
matches_pr = pd.read_csv(".\matches_premier.csv")
matches_pr.shape

(1222, 28)

In [90]:
#count the number of duplicated rows if any exist
num_duplicated = matches_pr.duplicated().sum()

print("Number of duplicated rows: ", num_duplicated)

Number of duplicated rows:  0


In [91]:
#Inspect the data and its columns
matches_pr.head()

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal


In [103]:
#The format of our datetime is wrong based on my inspection, the 'match report', 
#'unnamed: 0' and, 'Notes' columns serve no purpose
matches_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1222 non-null   datetime64[ns]
 1   Time        1222 non-null   object        
 2   Comp        1222 non-null   object        
 3   Round       1222 non-null   object        
 4   Day         1222 non-null   object        
 5   Venue       1222 non-null   object        
 6   Result      1222 non-null   object        
 7   GF          1222 non-null   float64       
 8   GA          1222 non-null   float64       
 9   Opponent    1222 non-null   object        
 10  xG          1222 non-null   float64       
 11  xGA         1222 non-null   float64       
 12  Poss        1222 non-null   float64       
 13  Attendance  1218 non-null   float64       
 14  Captain     1222 non-null   object        
 15  Formation   1222 non-null   object        
 16  Referee     1222 non-nul

In [93]:
#I need to convert the date column to a datetime
import datetime as dt
matches_pr['Date'] = matches_pr['Date'].apply(pd.to_datetime, errors='coerce')

In [94]:
#I also need to remove the unnamed column as it serves no purpose
matches_pr = matches_pr.drop('Unnamed: 0', axis=1)

In [104]:
matches_pr.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,4-3-3,Anthony Taylor,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,4-3-3,Darren England,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,4-3-3,Craig Pawson,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,4-3-3,Jarred Gillett,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,4-3-3,Robert Jones,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal


In [101]:
#The 'Match Report' and 'Notes' columns also serve no purpose
matches_pr = matches_pr.drop(['Match Report','Notes'], axis=1)

In [549]:
#To make sure all teams from the 2021 and 2022 season are accounted for. There should be 23 teams.
len(matches_pr['Team'].unique())

23

In [102]:
#To inspect the object type columns
matches_pr.select_dtypes(include=['object']).head()

,Time,Comp,Round,Day,Venue,Result,Opponent,Captain,Formation,Referee,Team
0,20:00,Premier League,Matchweek 1,Fri,Away,W,Crystal Palace,Martin Ødegaard,4-3-3,Anthony Taylor,Arsenal
1,15:00,Premier League,Matchweek 2,Sat,Home,W,Leicester City,Martin Ødegaard,4-3-3,Darren England,Arsenal
2,17:30,Premier League,Matchweek 3,Sat,Away,W,Bournemouth,Martin Ødegaard,4-3-3,Craig Pawson,Arsenal
3,17:30,Premier League,Matchweek 4,Sat,Home,W,Fulham,Martin Ødegaard,4-3-3,Jarred Gillett,Arsenal
4,19:30,Premier League,Matchweek 5,Wed,Home,W,Aston Villa,Martin Ødegaard,4-3-3,Robert Jones,Arsenal


In [105]:
#To inspect the number type columns
matches_pr.select_dtypes(include=[np.number]).head()

,GF,GA,xG,xGA,Poss,Attendance,Sh,SoT,Dist,FK,PK,PKatt,Season
0,2.0,0.0,1.0,1.2,44.0,25286.0,10.0,2.0,14.6,1.0,0.0,0.0,2022
1,4.0,2.0,2.7,0.5,50.0,60033.0,19.0,7.0,13.0,0.0,0.0,0.0,2022
2,3.0,0.0,1.3,0.3,57.0,10423.0,14.0,6.0,14.8,0.0,0.0,0.0,2022
3,2.0,1.0,2.6,0.8,71.0,60164.0,22.0,8.0,15.5,1.0,0.0,0.0,2022
4,2.0,1.0,2.4,0.4,59.0,60012.0,22.0,8.0,16.3,1.0,0.0,0.0,2022


## Data Analysis and Visualization
#### 1. Goal-scoring efficiency: I need to compare the number of shots taken by each team with the number of goals they have scored to determine the efficiency of their shooting. This will help me identify which teams are more effective at converting their chances into goals.

In [210]:
matches_pr.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Sh,SoT,Dist,FK,PK,PKatt,Season,Team,Sh_Accuracy
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Anthony Taylor,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal,20.0
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Darren England,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal,37.0
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Craig Pawson,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal,43.0
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Jarred Gillett,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal,36.0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Robert Jones,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal,36.0


In [185]:
#To do this, I would need a pivot table to show the shooting stats for each season
#first, my condition
matches_2022 = matches_pr[matches_pr['Season'] == 2022]
#Now my table, using the condition created
shootin_stats_22 = (matches_2022.pivot_table(['Sh','SoT','GA','GF','xG','xGA'],
                    index=['Team'],aggfunc= 'sum'))
shootin_stats_22
#This table shows me the shooting stats for each team in the 2022 season

,GA,GF,Sh,SoT,xG,xGA
Team,,,,,,
Arsenal,23.0,51.0,374.0,117.0,44.0,22.9
Aston Villa,38.0,28.0,259.0,90.0,27.1,35.8
Bournemouth,44.0,21.0,195.0,70.0,18.8,37.8
Brentford,30.0,37.0,232.0,88.0,32.5,29.3
Brighton and Hove Albion,29.0,39.0,314.0,118.0,35.9,26.4
Chelsea,23.0,23.0,271.0,88.0,28.0,29.1
Crystal Palace,31.0,21.0,242.0,81.0,21.8,32.8
Everton,30.0,17.0,230.0,75.0,23.9,37.0
Fulham,30.0,35.0,271.0,89.0,30.9,39.5


In [211]:
#To show what teams are clinal, I need an efficiency column
shootin_stats_22['Efficiency'] = shootin_stats_22['GF'] / shootin_stats_22['SoT']
shootin_stats_22

,GA,GF,Sh,SoT,xG,xGA,Efficiency
Team,,,,,,,
Arsenal,23.0,51.0,374.0,117.0,44.0,22.9,0.435897
Aston Villa,38.0,28.0,259.0,90.0,27.1,35.8,0.311111
Bournemouth,44.0,21.0,195.0,70.0,18.8,37.8,0.300000
Brentford,30.0,37.0,232.0,88.0,32.5,29.3,0.420455
Brighton and Hove Albion,29.0,39.0,314.0,118.0,35.9,26.4,0.330508
Chelsea,23.0,23.0,271.0,88.0,28.0,29.1,0.261364
Crystal Palace,31.0,21.0,242.0,81.0,21.8,32.8,0.259259
Everton,30.0,17.0,230.0,75.0,23.9,37.0,0.226667
Fulham,30.0,35.0,271.0,89.0,30.9,39.5,0.393258


In [562]:
# For the scatter plot of shots on target vs. goals scored in the 2022 season, I need my data points and a layout
trace_22 = go.Scatter(x=shootin_stats_22['SoT'], y=shootin_stats_22['GF'], mode='markers',
                   text=shootin_stats_22.index.get_level_values('Team'),
                   marker={'size': 40 * shootin_stats_22['Efficiency'],
                           'color':shootin_stats_22['Efficiency'],'colorscale': 'haline', 'showscale':True})
                            #To have the points vary based on efficiency

# create a layout for the plot
layout_22 = go.Layout(title='Goals Scored vs. Shots on Target (2022 Season)',
                   xaxis={'title': 'Shots on Target'},
                   yaxis={'title': 'Goals Scored'})

# create a figure object
shot_Stats_22 = go.Figure(data=[trace_22], layout=layout_22)
shot_Stats_22.show()
py.plot(shot_Stats_22, filename='ShotsvsGoals_Stats_22.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/1/'

In [216]:
# Now to create a table based on the 2021 season
matches_2021 = matches_pr[matches_pr['Season'] == 2021]
shootin_stats_21 = (matches_2021.pivot_table(['Sh','SoT','GA','GF','xG','xGA'],
                    index=['Team'],aggfunc= 'sum'))
shootin_stats_21['Efficiency'] = shootin_stats_21['GF'] / shootin_stats_21['SoT']
#To add the efficiency column to the 2021 table
shootin_stats_21
#This table shows me the shooting stats for each team in the 2021 season

,GA,GF,Sh,SoT,xG,xGA,Efficiency
Team,,,,,,,
Arsenal,48.0,61.0,581.0,186.0,60.4,45.6,0.327957
Aston Villa,54.0,52.0,461.0,159.0,43.9,49.2,0.327044
Brentford,56.0,48.0,436.0,141.0,45.8,48.4,0.340426
Brighton and Hove Albion,44.0,42.0,482.0,141.0,46.2,42.9,0.297872
Burnley,53.0,34.0,405.0,119.0,39.8,57.1,0.285714
Chelsea,33.0,76.0,583.0,200.0,63.2,32.9,0.380000
Crystal Palace,46.0,50.0,404.0,140.0,46.2,41.0,0.357143
Everton,66.0,43.0,430.0,130.0,41.4,55.2,0.330769
Leeds United,79.0,42.0,480.0,142.0,43.9,67.7,0.295775


In [563]:
# For the scatter plot of shots on target vs. goals scored in the 2021 season, 
# like the previous plot, I need my data points and a layout
trace_21 = go.Scatter(x=shootin_stats_21['SoT'], y=shootin_stats_21['GF'], mode='markers',
                   text=shootin_stats_21.index.get_level_values('Team'),
                   marker={'size': 35 * shootin_stats_21['Efficiency'],
                           'color':shootin_stats_21['Efficiency'],'colorscale': 'haline', 'showscale':True})
                              #To have the points vary based on efficiency
# To create a layout for the plot
layout_21 = go.Layout(title='Goals Scored vs. Shots on Target (2021 Season)',
                   xaxis={'title': 'Shots on Target'},
                   yaxis={'title': 'Goals Scored'})

# To show my chart
shot_Stats_21 = go.Figure(data=[trace_21], layout=layout_21)
shot_Stats_21.show()
py.plot(shot_Stats_21, filename='ShotsvsGoals_Stats_21.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/3/'

#### 2. Home vs. Away performance: I want to analyze the performance of each team at home and away to determine if there are any patterns or trends. This can help you identify which teams perform better at home or away.

In [232]:
#First we need our conditions for the 2021 season, home and then away
away_condition = ((matches_pr['Venue'] == 'Away')&(matches_pr['Season'] == 2021))
away_SS_21 = matches_pr[away_condition]

In [233]:
home_condition = ((matches_pr['Venue'] == 'Home')&(matches_pr['Season'] == 2021))
home_SS_21 = matches_pr[home_condition]

In [236]:
#Now we need to create our tables with this created conditions
away_shootin_stats_21 = (away_SS_21.pivot_table(['Sh','SoT','GA','GF','xG','xGA'],
                    index=['Team'],aggfunc= 'sum'))
away_shootin_stats_21['Efficiency'] = away_shootin_stats_21['GF'] / away_shootin_stats_21['SoT']

In [237]:
home_shootin_stats_21 = (home_SS_21.pivot_table(['Sh','SoT','GA','GF','xG','xGA'],
                    index=['Team'],aggfunc= 'sum'))
home_shootin_stats_21['Efficiency'] = home_shootin_stats_21['GF'] / home_shootin_stats_21['SoT']

In [564]:
# Now we just adjust the parameters of the previous scatter plot to show away numbers in the 2021 season
away_21 = go.Scatter(x=away_shootin_stats_21['SoT'], y=away_shootin_stats_21['GF'], mode='markers',
                   text=away_shootin_stats_21.index.get_level_values('Team'),
                   marker={'size': 35 * away_shootin_stats_21['Efficiency'],
                           'color':away_shootin_stats_21['Efficiency'],'colorscale': 'haline', 'showscale':True})
                              #To have the points vary based on efficiency
# create a layout for the plot
away_layout_21 = go.Layout(title='Goals Scored vs. Shots on Target away from Home (2021 Season)',
                   xaxis={'title': 'Shots on Target'},
                   yaxis={'title': 'Goals Scored'})

# To show our plot
shot_AwayStats_21 = go.Figure(data=[away_21], layout=away_layout_21)
shot_AwayStats_21.show()
py.plot(shot_AwayStats_21, filename='ShotsvsGoals_Away_Stats_21.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/5/'

In [565]:
# Now we just adjust the parameters of the previous scatter plot to show home numbers in the 2021 season
home_21 = go.Scatter(x=home_shootin_stats_21['SoT'], y=home_shootin_stats_21['GF'], mode='markers',
                   text=home_shootin_stats_21.index.get_level_values('Team'),
                   marker={'size': 35 * home_shootin_stats_21['Efficiency'],
                           'color':home_shootin_stats_21['Efficiency'],'colorscale': 'haline', 'showscale':True})
                              #To have the points vary based on efficiency
# create a layout for the plot
home_layout_21 = go.Layout(title='Goals Scored vs. Shots on Target at Home (2021 Season)',
                   xaxis={'title': 'Shots on Target'},
                   yaxis={'title': 'Goals Scored'})

# To show our plot
shot_HomeStats_21 = go.Figure(data=[home_21], layout=home_layout_21)
shot_HomeStats_21.show()
py.plot(shot_HomeStats_21, filename='ShotsvsGoals_Home_Stats_21.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/7/'

In [242]:
#Now we simply repeat the above steps for the 2022 season!
away_condition2 = ((matches_pr['Venue'] == 'Away')&(matches_pr['Season'] == 2022))
away_SS_22 = matches_pr[away_condition2]

In [243]:
home_condition2 = ((matches_pr['Venue'] == 'Home')&(matches_pr['Season'] == 2022))
home_SS_22 = matches_pr[home_condition2]

In [244]:
away_shootin_stats_22 = (away_SS_22.pivot_table(['Sh','SoT','GA','GF','xG','xGA'],
                    index=['Team'],aggfunc= 'sum'))
away_shootin_stats_22['Efficiency'] = away_shootin_stats_22['GF'] / away_shootin_stats_22['SoT']

In [245]:
home_shootin_stats_22 = (home_SS_22.pivot_table(['Sh','SoT','GA','GF','xG','xGA'],
                    index=['Team'],aggfunc= 'sum'))
home_shootin_stats_22['Efficiency'] = home_shootin_stats_22['GF'] / home_shootin_stats_22['SoT']

In [566]:
# For the 2022 season away stats
away_22 = go.Scatter(x=away_shootin_stats_22['SoT'], y=away_shootin_stats_22['GF'], mode='markers',
                   text=away_shootin_stats_22.index.get_level_values('Team'),
                   marker={'size': 35 * away_shootin_stats_22['Efficiency'],
                           'color':away_shootin_stats_22['Efficiency'],'colorscale': 'haline', 'showscale':True})
                              #To have the points vary based on efficiency
# create a layout for the plot
away_layout_22 = go.Layout(title='Goals Scored vs. Shots on Target away from Home (2022 Season)',
                   xaxis={'title': 'Shots on Target'},
                   yaxis={'title': 'Goals Scored'})

# create a figure object
shot_AwayStats_22 = go.Figure(data=[away_22], layout=away_layout_22)
shot_AwayStats_22.show()
py.plot(shot_AwayStats_22, filename='ShotsvsGoals_Away_Stats_22.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/9/'

In [567]:
# For the 2022 season home stats
home_22 = go.Scatter(x=home_shootin_stats_22['SoT'], y=home_shootin_stats_22['GF'], mode='markers',
                   text=home_shootin_stats_22.index.get_level_values('Team'),
                   marker={'size': 35 * home_shootin_stats_22['Efficiency'],
                           'color':home_shootin_stats_22['Efficiency'], 'colorscale': 'haline', 'showscale':True})
                              #To have the points vary based on efficiency
# create a layout for the plot
home_layout_22 = go.Layout(title='Goals Scored vs. Shots on Target at Home (2022 Season)',
                   xaxis={'title': 'Shots on Target'},
                   yaxis={'title': 'Goals Scored'})

# create a figure object
shot_HomeStats_22 = go.Figure(data=[home_22], layout=home_layout_22)
shot_HomeStats_22.show()
py.plot(shot_HomeStats_22, filename='ShotsvsGoals_Home_Stats_22.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/11/'

#### 3. Do referees award fouls too often? What referees award the most free kicks or penalties? What's the trend over the past two seasons? 

In [250]:
matches_pr.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Sh,SoT,Dist,FK,PK,PKatt,Season,Team,Sh_Accuracy
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Anthony Taylor,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal,20.0
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Darren England,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal,37.0
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Craig Pawson,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal,43.0
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Jarred Gillett,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal,36.0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Robert Jones,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal,36.0


In [253]:
matches_pr.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation',
       'Referee', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt', 'Season', 'Team',
       'Sh_Accuracy'],
      dtype='object')

In [258]:
matches_pr[['Referee','Team','Season','Result','FK','PK','PKatt']].head(50)

,Referee,Team,Season,Result,FK,PK,PKatt
0,Anthony Taylor,Arsenal,2022,W,1.0,0.0,0.0
1,Darren England,Arsenal,2022,W,0.0,0.0,0.0
2,Craig Pawson,Arsenal,2022,W,0.0,0.0,0.0
3,Jarred Gillett,Arsenal,2022,W,1.0,0.0,0.0
4,Robert Jones,Arsenal,2022,W,1.0,0.0,0.0
5,Paul Tierney,Arsenal,2022,L,1.0,0.0,0.0
6,David Coote,Arsenal,2022,W,0.0,0.0,0.0
7,Anthony Taylor,Arsenal,2022,W,1.0,0.0,0.0
8,Michael Oliver,Arsenal,2022,W,0.0,1.0,1.0
9,Chris Kavanagh,Arsenal,2022,W,1.0,0.0,0.0


In [352]:
referees = matches_pr.pivot_table(['Result','FK','PK','PKatt'], index=['Referee'], aggfunc= ['sum'])
#To make the freekicks column more understandable, I changed the name
#referees.rename(columns={'FK': 'FKatt'}, inplace=True)
len(referees)

25

In [342]:
# Here I am getting the 25th and 75th quartiles
#round(referees['sum'][['PKatt']].describe())
round(referees['sum'][['FK']].describe())

,PKatt
count,25.0
mean,6.0
std,4.0
min,0.0
25%,4.0
50%,6.0
75%,8.0
max,13.0


In [568]:
# In order to get a more accurate distribution, I created a condition filtering the out data 
# outside the 25th to 75th quatile range for Penalty kicks
condition_pk = ((referees['sum']['PKatt']>= 4) & (referees['sum']['PKatt'] <= 8))
y = referees[condition_pk]['sum']['PKatt']
x = referees.index


referee_pk =  go.Figure([go.Bar(x=x, y=y, marker={
                        'color':y,'colorscale': 'purples', 'showscale': True})])
referee_pk.update_layout(
            title='Penalties Awarded',
                xaxis_title = 'Penalties',
                yaxis_title = 'Referees')
referee_pk.show()
py.plot(referee_pk, filename='Referee_Penalty_Kicks_Analysis.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/13/'

In [569]:
condition_fk = ((referees['sum']['FK']>= 14) & (referees['sum']['FK'] <= 28))
y = referees[condition_fk]['sum']['FK']
x = referees.index

referee_fk = go.Figure([go.Bar(x=x, y=y, 
                marker={'color': y, 'colorscale': 'purples', 
                        'showscale': True})])

referee_fk.update_layout(
            title='Free Kicks Awarded',
                xaxis_title = 'Free Kicks',
                yaxis_title = 'Referees')
referee_fk.show()
py.plot(referee_fk, filename='Referee_FreeKicks_Analysis.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/15/'

In [354]:
matches_pr[['Referee','Team','Season','Result','FK','PK','PKatt']].head(50)
#What's the trend over the past two seasons?
#matches_pr.columns
#matches_pr.groupby(['Team','Season'])['FK','PKatt'].sum()
matches_pr.pivot_table(['FK','PKatt'],index=['Team','Season'], aggfunc= 'sum')
# Looking at this table, I am unable to really answer this question as 
# I lack the necesary data to continue exploring this.2022 is an incomplete season

FK  PKatt
Team                     Season             
Arsenal                  2021    22.0    8.0
                         2022    10.0    2.0
Aston Villa              2021    20.0    3.0
                         2022    10.0    2.0
Bournemouth              2022    10.0    0.0
Brentford                2021    11.0    6.0
                         2022     7.0    5.0
Brighton and Hove Albion 2021    15.0    7.0
                         2022     8.0    3.0
Burnley                  2021    14.0    2.0
Chelsea                  2021    26.0    9.0
                         2022     6.0    2.0
Crystal Palace           2021    13.0    8.0
                         2022    12.0    2.0
Everton                  2021    18.0    7.0
                         2022     5.0    1.0
Fulham                   2022     9.0    7.0
Leeds United             2021    13.0    5.0
                         2022    13.0    2.0
Leicester City           2021    14.0    2.0
                         2022     9.0    2.0
Liverpool                2021    13.0    8.0
                         2022     5.0    0.0
Manchester City          2021    17.0    9.0
                         2022    11.0    5.0
Manchester United        2021    19.0    5.0
                         2022    12.0    2.0
Newcastle United         2021    15.0    3.0
                         2022    10.0    3.0
Norwich City             2021    18.0    3.0
Nottingham Forest        2022     9.0    2.0
Southampton              2021    24.0    4.0
                         2022    10.0    1.0
Tottenham Hotspur        2021    16.0    5.0
                         2022    10.0    3.0
Watford                  2021    24.0    2.0
West Ham United          2021    13.0    6.0
                         2022     9.0    6.0
Wolverhampton Wanderers  2021    13.0    1.0
                         2022    12.0    2.0

#### 4. What role does possession play in the team? Is it a deciding factor in the finishing position of teams?

In [458]:
shootin_stats_21.head()
matches_pr[['Team','Poss']].value_counts()
poss_21 = matches_2021.pivot_table('Poss', index= 'Team', aggfunc= 'mean')
poss_22 = matches_2022.pivot_table('Poss', index= 'Team', aggfunc= 'mean')
possession = matches_pr.groupby(['Team'])[['Poss']].mean()
#poss['Poss_diff'] = poss['Season']['2022'] - poss['Season']['2021']
possession['Poss']

Team
Arsenal                     54.967213
Aston Villa                 47.000000
Bournemouth                 39.434783
Brentford                   44.360656
Brighton and Hove Albion    56.016667
Burnley                     40.157895
Chelsea                     60.688525
Crystal Palace              48.672131
Everton                     42.065574
Fulham                      46.291667
Leeds United                51.295082
Leicester City              51.245902
Liverpool                   61.633333
Manchester City             66.564516
Manchester United           52.919355
Newcastle United            44.557377
Norwich City                42.868421
Nottingham Forest           39.478261
Southampton                 46.409836
Tottenham Hotspur           51.338710
Watford                     40.710526
West Ham United             46.360656
Wolverhampton Wanderers     49.622951
Name: Poss, dtype: float64

In [570]:
# To show the possession stats of the different teams


poss_st = go.Scatter(y=possession['Poss'],x=possession.index, mode='markers',
                    marker={'size': .35 * possession['Poss'], 
                            'color':possession['Poss'], 'colorscale':'Viridis', 'showscale':True})
                    #To have the points vary in size based on possession
# create a layout for the plot
poss_layout = go.Layout(title='Possession Stats',
                   yaxis={'title': 'Possession'},
                   xaxis={'title': 'Teams'})

# create a figure object
Summarized_possesion = go.Figure(data=[poss_st], layout=poss_layout)
Summarized_possesion.show()
py.plot(Summarized_possesion, filename='Summarized_Possession.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/17/'

In [572]:
#To do a side by side comparison of the possession stats between two seasons
poss_21_ch = go.Bar(x=poss_21.index, y=poss_21['Poss'],name= 'Possession Stats 21',marker=dict(color='#00BFFF'),opacity=0.5)
poss_22_ch = go.Bar(x=poss_21.index, y=poss_22['Poss'],name= 'Possession Stats 22',marker=dict(color='#A52A2A'),opacity=0.5)

data = [poss_22_ch,poss_21_ch]

layout = go.Layout(barmode='group',title= 'Possession Statistics',
                   xaxis_title = 'Teams',
                yaxis_title = 'Possession')
possession_ch = go.Figure(data=data, layout=layout)
possession_ch.show()
py.plot(possession_ch, filename='Possession_Stats.html', auto_open=False)

'https://plotly.com/~Kvothe_v1/19/'